In [1]:
import pandas as pd 
import numpy as np 
import pingouin as pg 

/volatile/miniconda3/envs/ohbm/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
# read in the metric files saved as csv
controls = pd.read_csv('../../DerivedData/extracted_diffusion_metrics_control_group_mergedLR.csv', index_col=0)
preterms = pd.read_csv('../../DerivedData/extracted_diffusion_metrics_preterm_group_mergedLR.csv', index_col=0)

### first get all regions pairs, metrics to be evaluated 
tract_names = np.unique(np.array([tract.split('_')[0] for tract in controls.columns[2:]]))
print('Number of evaluated bundles: {}'.format(len(tract_names)))
metrics = np.unique(np.array([tract.partition('_')[-1] for tract in controls.columns[2:]]))

### get matching + extreme/moderate grouping 
extreme_and_match = pd.read_csv('../../DerivedData/extreme_pairs_mean_diffusion_metrics_over_WM.csv', index_col=0)[['preterm_ID', 'matched_ID']]
moderate_and_match = pd.read_csv('../../DerivedData/moderate_pairs_mean_diffusion_metrics_over_WM.csv', index_col=0)[['preterm_ID', 'matched_ID']]
        

Number of evaluated bundles: 15


# Group comparison 
### 1. All bundles - independently of tracts 

Moderate vs Control  <br>
Extreme vs Control<br>
Extreme Control vs Moderate Control <br>
Extreme preterm vs Moderate Preterm<br>

In [3]:
#### EXTREME VS CONTROL
### create new dfs in a loop to enfore subject matching for paired comparisons 

group_results = pd.DataFrame()

for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]

    extreme_df = pd.DataFrame()
    ext_control_df = pd.DataFrame()

    j = 0
    for  i, row in extreme_and_match.iterrows():
        extreme_df.loc[j,'subject_id'] = row.preterm_ID
        ext_control_df.loc[j,'subject_id'] = row.matched_ID
    
        for col in metric_subset:
            extreme_df.loc[j,col] = preterms[preterms.subject_id == row.preterm_ID][col].values
            ext_control_df.loc[j,col] = controls[controls.subject_id == row.matched_ID][col].values

        j = j+1
    
    ### first run over the entire bundle set 

    extreme_vals = []
    control_vals = []
    
    for col in extreme_df.columns[1:]:
        extreme_vals.append(extreme_df[col].values)
        control_vals.append(ext_control_df[col].values)

    extreme_vals = np.array(extreme_vals).ravel()
    control_vals = np.array(control_vals).ravel()
    
    test = pg.ttest(extreme_vals, control_vals, paired=True)
    
    group_results.loc[i_met, 'metric'] = metric
    group_results.loc[i_met, 'T'] = test['T'].values[0]
    group_results.loc[i_met, 'p-val'] = test['p-val'].values[0]
    group_results.loc[i_met,  'CI95%'] = str(test['CI95%'].values[0])


group_results['p-val'] = group_results['p-val'].astype(np.float64)
#group_results['p-val'] = np.round(group_results['p-val'],6)
reject, pvals_corr = pg.multicomp(group_results['p-val'].values, method='fdr_bh')
group_results['p-val_fdr_bh_corrected'] = pvals_corr


In [4]:
group_results

,metric,T,p-val,CI95%,p-val_fdr_bh_corrected
0,FA,-11.474342,3.507166e-27,[-0.02 -0.01],3.507166e-27
1,AD,22.486554,1.232078e-77,[0. 0.],3.696234e-77
2,RD,20.984819,2.241150e-70,[0. 0.],3.361725e-70
3,MD,22.730414,8.157902e-79,[0. 0.],4.894741e-78
4,ODI,-13.843303,4.620690e-37,[-0.02 -0.01],5.544828e-37
5,NDI,-21.760890,3.979649e-74,[-0.03 -0.02],7.959299e-74


In [5]:
#### Moderate VS CONTROL
### create new dfs in a loop to enfore subject matching for paired comparisons 

group_results = pd.DataFrame()

for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]

    extreme_df = pd.DataFrame()
    ext_control_df = pd.DataFrame()

    j = 0
    for  i, row in moderate_and_match.iterrows():
        extreme_df.loc[j,'subject_id'] = row.preterm_ID
        ext_control_df.loc[j,'subject_id'] = row.matched_ID
    
        for col in metric_subset:
            extreme_df.loc[j,col] = preterms[preterms.subject_id == row.preterm_ID][col].values
            ext_control_df.loc[j,col] = controls[controls.subject_id == row.matched_ID][col].values

        j = j+1
    
    ### first run over the entire bundle set 

    extreme_vals = []
    control_vals = []
    
    for col in extreme_df.columns[1:]:
        extreme_vals.append(extreme_df[col].values)
        control_vals.append(ext_control_df[col].values)

    extreme_vals = np.array(extreme_vals).ravel()
    control_vals = np.array(control_vals).ravel()
    
    test = pg.ttest(extreme_vals, control_vals, paired=True)
    
    group_results.loc[i_met, 'metric'] = metric
    group_results.loc[i_met, 'T'] = test['T'].values[0]
    group_results.loc[i_met, 'p-val'] = test['p-val'].values[0]
    group_results.loc[i_met,  'CI95%'] = str(test['CI95%'].values[0])


group_results['p-val'] = group_results['p-val'].astype(np.float64)
#group_results['p-val'] = np.round(group_results['p-val'],6)
reject, pvals_corr = pg.multicomp(group_results['p-val'].values, method='fdr_bh')
group_results['p-val_fdr_bh_corrected'] = pvals_corr

In [6]:
group_results

,metric,T,p-val,CI95%,p-val_fdr_bh_corrected
0,FA,-2.105426,3.589480e-02,[-0.01 -0. ],3.589480e-02
1,AD,6.027737,3.863603e-09,[0. 0.],2.318162e-08
2,RD,5.487228,7.372803e-08,[0. 0.],1.474561e-07
3,MD,5.873085,9.185162e-09,[0. 0.],2.755549e-08
4,ODI,-4.867862,1.642939e-06,[-0.01 -0. ],2.464408e-06
5,NDI,-3.779636,1.816848e-04,[-0.01 -0. ],2.180217e-04


In [7]:
#### Extreme control VS Moderate control
from scipy.stats import ttest_ind

group_results = pd.DataFrame()

moderate_vals = controls[controls['subject_id'].isin(moderate_and_match.matched_ID.values)].copy()
extreme_vals = controls[controls['subject_id'].isin(extreme_and_match.matched_ID.values)].copy()


for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]
    
    
    moderate_m = moderate_vals[metric_subset]
    extreme_m = extreme_vals[metric_subset]
    
    ex_ar = []
    mod_ar = []
    
    for col in metric_subset:
        ex_ar.append(extreme_m[col].values)
        mod_ar.append(moderate_m[col].values)
    ex_ar = np.array(ex_ar).ravel()
    mod_ar = np.array(mod_ar).ravel()
    
    test = pg.ttest(ex_ar, mod_ar, paired=False, correction='auto')
    
    #print(ttest_ind(ex_ar, mod_ar))
    group_results.loc[i_met, 'metric'] = metric
    group_results.loc[i_met, 'T'] = test['T'].values[0]
    group_results.loc[i_met, 'p-val'] = test['p-val'].values[0]
    group_results.loc[i_met,  'CI95%'] = str(test['CI95%'].values[0])


group_results['p-val'] = group_results['p-val'].astype(np.float64)
#group_results['p-val'] = np.round(group_results['p-val'],6)
reject, pvals_corr = pg.multicomp(group_results['p-val'].values, method='fdr_bh')
group_results['p-val_fdr_bh_corrected'] = pvals_corr    

In [8]:
group_results

,metric,T,p-val,CI95%,p-val_fdr_bh_corrected
0,FA,1.513856,0.130438,[-0. 0.01],0.130438
1,AD,-2.073754,0.038419,[-0. -0.],0.046103
2,RD,-2.473887,0.013570,[-0. -0.],0.032007
3,MD,-2.630754,0.008685,[-0. -0.],0.032007
4,ODI,2.306275,0.021338,[0. 0.01],0.032007
5,NDI,2.358222,0.018599,[0. 0.01],0.032007


In [9]:
#### Extreme preterm VS Moderate preterm
from scipy.stats import ttest_ind

group_results = pd.DataFrame()

moderate_vals = preterms[preterms['subject_id'].isin(moderate_and_match.preterm_ID.values)].copy()
extreme_vals = preterms[preterms['subject_id'].isin(extreme_and_match.preterm_ID.values)].copy()


for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]
    
    
    moderate_m = moderate_vals[metric_subset]
    extreme_m = extreme_vals[metric_subset]
    
    ex_ar = []
    mod_ar = []
    
    for col in metric_subset:
        ex_ar.append(extreme_m[col].values)
        mod_ar.append(moderate_m[col].values)
    ex_ar = np.array(ex_ar).ravel()
    mod_ar = np.array(mod_ar).ravel()
    
    test = pg.ttest(ex_ar, mod_ar, paired=False, correction='auto')
    
    #print(ttest_ind(ex_ar, mod_ar))
    group_results.loc[i_met, 'metric'] = metric
    group_results.loc[i_met, 'T'] = test['T'].values[0]
    group_results.loc[i_met, 'p-val'] = test['p-val'].values[0]
    group_results.loc[i_met,  'CI95%'] = str(test['CI95%'].values[0])


group_results['p-val'] = group_results['p-val'].astype(np.float64)
#group_results['p-val'] = np.round(group_results['p-val'],6)
reject, pvals_corr = pg.multicomp(group_results['p-val'].values, method='fdr_bh')
group_results['p-val_fdr_bh_corrected'] = pvals_corr   

In [10]:
group_results

,metric,T,p-val,CI95%,p-val_fdr_bh_corrected
0,FA,-1.586925,0.112915,[-0.01 0. ],0.112915
1,AD,4.331495,0.000017,[0. 0.],0.000100
2,RD,3.257178,0.001171,[0. 0.],0.001757
3,MD,3.879416,0.000113,[0. 0.],0.000330
4,ODI,-2.077877,0.038020,[-0.01 -0. ],0.045624
5,NDI,-3.785321,0.000165,[-0.02 -0.01],0.000330


### 2. Independent tract comparison 

Moderate vs Control  <br>
Extreme vs Control<br>
Extreme Control vs Moderate Control <br>
Extreme preterm vs Moderate Preterm<br>

In [11]:
#### EXTREME VS CONTROL
### create new dfs in a loop to enfore subject matching for paired comparisons 

group_results = pd.DataFrame()
T_results = pd.DataFrame()

for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]

    extreme_df = pd.DataFrame()
    ext_control_df = pd.DataFrame()

    j = 0
    for  i, row in extreme_and_match.iterrows():
        extreme_df.loc[j,'subject_id'] = row.preterm_ID
        ext_control_df.loc[j,'subject_id'] = row.matched_ID
    
        for col in metric_subset:
            extreme_df.loc[j,col] = preterms[preterms.subject_id == row.preterm_ID][col].values
            ext_control_df.loc[j,col] = controls[controls.subject_id == row.matched_ID][col].values

        j = j+1
    
   
    group_results.loc[i_met, 'metric'] = metric
    T_results.loc[i_met, 'metric'] = metric
    
    for col in metric_subset:
        new_col = col.split('_')[0]

        test = pg.ttest(extreme_df[col].values, ext_control_df[col].values, paired=True)
        p = "{:.12f}".format(test['p-val'].values[0])
        t =  "{:.12f}".format(test['T'].values[0])
        T_results.loc[i_met, new_col] = t
        group_results.loc[i_met, new_col] = p
       
        #print(metric, col)
        #print(p)


group_results[group_results.columns[1:]] = group_results[group_results.columns[1:]].astype(np.float64)

pre_p = group_results[group_results.columns[1:]].values.ravel()
reject, pvals_corr = pg.multicomp(pre_p, method='fdr_bh')
pvals_corr = pvals_corr.reshape(np.shape(group_results[group_results.columns[1:]].values))

group_results[group_results.columns[1:]] = pvals_corr


In [14]:
group_results

,metric,M1-Brainstem,M1-Caud,M1-Lenti,M1-Thalfus,M1L-M1R,Paracentral-Brainstem,Paracentral-Caud,Paracentral-Lenti,Paracentral-Thalfus,S1-Brainstem,S1-Caud,S1-Lenti,S1-M1,S1-Thalfus,S1L-S1R
0,FA,0.067925,0.937556,0.000360,0.073937,1.823723e-06,0.001520,0.001117,0.000430,0.001466,0.563632,8.234243e-01,9.446837e-05,1.020226e-05,1.521868e-02,8.620907e-07
1,AD,0.000032,0.000011,0.000007,0.000002,1.636654e-05,0.000106,0.000018,0.000002,0.000098,0.000013,6.340500e-08,6.798808e-07,5.520975e-07,2.028300e-07,5.205842e-06
2,RD,0.000057,0.007011,0.000005,0.000071,6.330000e-08,0.000002,0.000003,0.000003,0.000038,0.000608,1.184922e-04,2.298132e-06,2.028300e-07,7.654763e-06,1.035000e-08
3,MD,0.000012,0.001045,0.000004,0.000013,7.327800e-08,0.000002,0.000003,0.000002,0.000035,0.000032,5.140793e-06,1.468116e-06,2.028300e-07,1.823723e-06,1.035000e-08
4,ODI,0.004705,0.000002,0.002402,0.000032,1.170797e-03,0.080067,0.002729,0.016422,0.007169,0.000044,1.316300e-05,2.402922e-06,1.170797e-03,1.823723e-06,3.808497e-01
5,NDI,0.000259,0.000836,0.000003,0.000013,2.717345e-07,0.000004,0.000002,0.000002,0.000015,0.002887,1.180649e-05,2.314137e-06,2.717345e-07,2.167991e-06,8.551500e-08


In [13]:
group_results.T.to_csv('extreme_vs_control_per_tract_new.csv')
T_results.T.to_csv('extreme_vs_control_per_tract_T.csv')

In [214]:
#### MODERATE VS CONTROL
### create new dfs in a loop to enfore subject matching for paired comparisons 

group_results = pd.DataFrame()
T_results = pd.DataFrame()

for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]

    extreme_df = pd.DataFrame()
    ext_control_df = pd.DataFrame()

    j = 0
    for  i, row in moderate_and_match.iterrows():
        extreme_df.loc[j,'subject_id'] = row.preterm_ID
        ext_control_df.loc[j,'subject_id'] = row.matched_ID
    
        for col in metric_subset:
            extreme_df.loc[j,col] = preterms[preterms.subject_id == row.preterm_ID][col].values
            ext_control_df.loc[j,col] = controls[controls.subject_id == row.matched_ID][col].values

        j = j+1
    
   
    group_results.loc[i_met, 'metric'] = metric
    
    for col in metric_subset:
        new_col = col.split('_')[0]

        test = pg.ttest(extreme_df[col].values, ext_control_df[col].values, paired=True)
        p = "{:.12f}".format(test['p-val'].values[0])
        t =  "{:.12f}".format(test['T'].values[0])
        T_results.loc[i_met, new_col] = t
        group_results.loc[i_met, new_col+'_p'] = p 


group_results[group_results.columns[1:]] = group_results[group_results.columns[1:]].astype(np.float64)

pre_p = group_results[group_results.columns[1:]].values.ravel()
reject, pvals_corr = pg.multicomp(pre_p, method='fdr_bh')
pvals_corr = pvals_corr.reshape(np.shape(group_results[group_results.columns[1:]].values))

group_results[group_results.columns[1:]] = pvals_corr

In [215]:
group_results.T.to_csv('moderate_vs_control_per_tract.csv')
T_results.T.to_csv('moderate_vs_control_per_tract_T.csv')

In [206]:
#### CONTROL vs CONTROL
### create new dfs in a loop to enfore subject matching for paired comparisons 

group_results = pd.DataFrame()


moderate_vals = controls[controls['subject_id'].isin(moderate_and_match.matched_ID.values)].copy()
extreme_vals = controls[controls['subject_id'].isin(extreme_and_match.matched_ID.values)].copy()


for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]

   
    group_results.loc[i_met, 'metric'] = metric
    
    for col in metric_subset:
        new_col = col.split('_')[0]

        test = pg.ttest(moderate_vals[col].values, extreme_vals[col].values, paired=False, correction='auto')
        p = "{:.12f}".format(test['p-val'].values[0])
        t =  "{:.12f}".format(test['T'].values[0])
        group_results.loc[i_met, new_col+'_T'] = t
        group_results.loc[i_met, new_col+'_p'] = p
        #print(metric, col)
        #print(p)


group_results[group_results.columns[1:]] = group_results[group_results.columns[1:]].astype(np.float64)

pre_p = group_results[group_results.columns[1:]].values.ravel()
reject, pvals_corr = pg.multicomp(pre_p, method='fdr_bh')
pvals_corr = pvals_corr.reshape(np.shape(group_results[group_results.columns[1:]].values))
group_results[group_results.columns[1:]] = pvals_corr

In [207]:
group_results.T.to_csv('control_vs_control_per_tract.csv')

In [208]:
#### PRETERM vs PRETERM
### create new dfs in a loop to enfore subject matching for paired comparisons 

group_results = pd.DataFrame()


moderate_vals = preterms[preterms['subject_id'].isin(moderate_and_match.preterm_ID.values)].copy()
extreme_vals = preterms[preterms['subject_id'].isin(extreme_and_match.preterm_ID.values)].copy()


for i_met, metric in enumerate(['FA', 'AD', 'RD', 'MD', 'ODI', 'NDI']):
    metric_subset= [tract + '_' + metric for tract in tract_names]

   
    group_results.loc[i_met, 'metric'] = metric
    
    for col in metric_subset:
        new_col = col.split('_')[0]

        test = pg.ttest(moderate_vals[col].values, extreme_vals[col].values, paired=False, correction='auto')
        p = "{:.12f}".format(test['p-val'].values[0])
        t =  "{:.12f}".format(test['T'].values[0])
        group_results.loc[i_met, new_col+'_T'] = t
        group_results.loc[i_met, new_col+'_p'] = p
        
        #print(metric, col)
        #print(p)


group_results[group_results.columns[1:]] = group_results[group_results.columns[1:]].astype(np.float64)

pre_p = group_results[group_results.columns[1:]].values.ravel()
reject, pvals_corr = pg.multicomp(pre_p, method='fdr_bh')
pvals_corr = pvals_corr.reshape(np.shape(group_results[group_results.columns[1:]].values))
group_results[group_results.columns[1:]] = pvals_corr

In [209]:
group_results.T.to_csv('preterm_vs_preterm_per_tract.csv')